# 3. 队列 Queue

## 3.1 队列的定义
队列是一种**有次序**的数据集合，特征是：新数据项的添加总发生在一端（**尾端**），移除总发生在另一端（**首端**），符合FIFO(First in first out)先进先出原则。队列只有一个入口一个出口，不允许从中间插入或取出。

对比：栈是后进先出，队列是先进先出。

举例：排队；打印机依次打印（先到先服务）；进程调度；键盘缓冲（字符输入和显示的顺序一致）。

In [10]:
class Queue:
    def __init__(self):
        self.item = []
    def isEmpty(self):
        return self.item == []
    def size(self):
        return len(self.item)
    def enqueue(self, data):
        self.item.append(data)  # 在尾部添加 O(1)
    def dequeue(self):
        return self.item.pop(0) # 在头部删除 O(n)
    def peek(self):
        return self.item[0]

if __name__ == "__main__": 
    q = Queue()
    print(q.isEmpty())  # True
    q.enqueue(1)
    q.enqueue(2)
    q.enqueue(3)
    print(q.size())     # 3
    print(q.peek())     # 1
    print(q.dequeue())  # 1
    print(q.size())     # 2
    print(q.isEmpty())  # False

True
3
1
1
2
False


## 3.2 队列的应用

### 3.2.1 热土豆算法
热土豆算法（Hot Potato）也被称为约瑟夫问题（Josephus problem）的一个变体。
游戏规则：一群人围成一圈，传递一个物品（比如土豆），传递固定次数后，拿到物品的人被淘汰。
然后从下一个人重新开始传递，重复这个过程，直到只剩下一个人。

步骤：
1. 将所有人按顺序放入队列。
2. 从队列头部开始，每次将队头的人出队，然后立即入队到队尾，模拟传递一次。
3. 传递固定次数（比如num）后，此时队头的人就是拿着土豆的人，将其移除（不再入队）。
4. 重复步骤2和3，直到队列中只剩下一个人。

In [24]:
def hotPotato(namelist, num):
    '''
    :param namelist: 名字列表
    :param num: 每一轮的传递次数
    '''
    q = Queue()
    for name in namelist:
        q.enqueue(name)
    while q.size() > 1:
        for i in range(num):
            q.enqueue(q.dequeue())  # 将队首元素出队后再入队，实现传递
        q.dequeue() # 出队，表示被淘汰

    return q.dequeue()  # 返回最后剩下的名字

print(hotPotato(["Bill","David","Susan","Jane","Kent","Brad"],3))

Kent


### 3.2.2 模拟打印任务
多人共享一台打印机，先到先服务执行打印任务，求问打印系统的容量有多大。

分析涉及到的对象：
1. 打印任务属性：提交时间、页数；
2. 打印队列属性：FIFO；
3. 打印机属性：打印速度、是否忙碌；

生成打印过程：
1. 确定生成概率：每秒生成作业的概率；
2. 确定打印页数：假定1-20页之间均匀分布；

实施打印过程：
1. 当前打印作业；
2. 打印结束倒计时：新作业开始打印时开始倒计时，回0表示打印完毕；

模拟实践：
1. 统一的时间框架（秒为单位），设定结束时间；
2. 同步所有过程：在一个时间单位里对**生成打印任务**和**实施打印**两个过程各处理一次；

In [26]:
import random

class Printer:
    def __init__(self, ppm):
        self.pagerate = ppm  # 每分钟打印页数
        self.currentTask = None  # 当前任务
        self.timeRemaining = 0  # 当前任务剩余时间

    def tick(self):
        if self.currentTask != None:
            self.timeRemaining = self.timeRemaining - 1 # 每次调用tick表示过了一秒钟
            if self.timeRemaining <= 0:
                self.currentTask = None

    def busy(self):
        return self.currentTask != None # 如果有任务正在打印则返回True表示在忙

    def startNext(self, newtask):
        self.currentTask = newtask
        self.timeRemaining = newtask.getPages() * 60 / self.pagerate

In [27]:
class Task:
    def __init__(self, time):
        self.timestamp = time  # 任务创建的时间
        self.pages = random.randint(1,20)  # 任务的页数，1到20页之间随机

    def getStamp(self):
        return self.timestamp

    def getPages(self):
        return self.pages

    def waitTime(self, currenttime):
        return currenttime - self.timestamp

# 当前这一秒是否有新任务产生
def newPrintTask():
    num = random.randrange(1,181)  # 1到180之间随机数 表示生成概率1/180
    return num == 180  # 以1/180的概率返回True

In [35]:
def simulation(numSeconds, pagesPerMinute):
    labprinter = Printer(pagesPerMinute)
    printQueue = Queue()
    waitingtimes = []

    for currentSecond in range(numSeconds):
        if newPrintTask():
            task = Task(currentSecond)  # 创建新任务
            printQueue.enqueue(task)    # 将任务加入打印队列

        if (not labprinter.busy()) and (not printQueue.isEmpty()):  # 如果打印机空闲且有任务等待
            nexttask = printQueue.dequeue() # 取出下一个任务
            waitingtimes.append(nexttask.waitTime(currentSecond))   # 计算等待时间并记录
            labprinter.startNext(nexttask)  # 开始打印下一个任务

        labprinter.tick()   # 让打印机工作一秒钟

    averageWait = sum(waitingtimes) / len(waitingtimes) if waitingtimes else 0
    print(f"Average Wait {averageWait:6.2f} secs {printQueue.size()} tasks remaining.")

for i in range(10):
    simulation(3600, 10)  # 模拟一小时，打印机打印速度为每分钟10页

Average Wait  35.78 secs 0 tasks remaining.
Average Wait   1.00 secs 0 tasks remaining.
Average Wait  32.09 secs 0 tasks remaining.
Average Wait  27.35 secs 0 tasks remaining.
Average Wait  17.52 secs 0 tasks remaining.
Average Wait  12.12 secs 0 tasks remaining.
Average Wait  11.95 secs 1 tasks remaining.
Average Wait   8.53 secs 0 tasks remaining.
Average Wait  11.22 secs 1 tasks remaining.
Average Wait  20.32 secs 0 tasks remaining.


## 3.3 双端队列 Deque
双端队列有首尾两端，但是deque中数据项既可以从队首加入，也可以从队尾加入，数据项也可以从两端移除。某种程度上，双端队列集成了栈和队列的能力。Deque并不具有内在的LIFO和FIFO的特性。

In [40]:
class Deque:
    def __init__(self):
        self.item = []
    def isEmpty(self):
        return self.item == []
    def size(self):
        return len(self.item)
    def addFront(self, data):
        self.item.insert(0, data)  # 在头部添加 O(n)
    def addRear(self, data):
        self.item.append(data)     # 在尾部添加 O(1)
    def removeFront(self):
        return self.item.pop(0)    # 在头部删除 O(n)
    def removeRear(self):
        return self.item.pop()      # 在尾部删除 O(1)
    def getItems(self):
        return self.item

if __name__ == "__main__":
    d = Deque()
    print(d.isEmpty())  # True
    d.addRear(1)    # 在尾部添加1
    d.addRear(2)    # 在尾部添加2
    d.addFront(3)   # 在头部添加3
    print(d.getItems()) # [3, 1, 2]
    print(d.size())     # 3
    print(d.removeFront()) # 3
    print(d.removeRear())  # 2
    print(d.size())        # 1

True
[3, 1, 2]
3
3
2
1


## 3.4 双端队列的应用

### 3.4.1 回文词判断
思路：先将需要判断的词从队尾加入deque，再从两端同时移除字符，判断字符是否相同，直到deque剩下0个或1个字符。

In [42]:
def palchecker(string):

    d = Deque()
    for char in string:
        d.addRear(char)  # 将字符串的每个字符从尾端添加到双端队列中

    stillEqual = True
    while d.size() > 1 and stillEqual:
        first = d.removeFront()  # 从头部移除一个字符
        last = d.removeRear()    # 从尾部移除一个字符
        if first != last:
            stillEqual = False

    return stillEqual

print(palchecker("lsdkjfskf"))  # False
print(palchecker("radar"))      # True

False
True
